# Cecklist

## Frame the problem and look at big picture
[X] Define the objective.

    - predict winners of each game in world cup

[X] What are the current solutions if any

    - Many

[X] How should performance be measures

    - How well predictions come to fruition. 

[X] List the assumptions you or others have made so far.

    - spi will predict winner
    - gather SPI of starters for each team
    - use cummulative SPI for each team 

[ ] Verify assumptions if possible



## Get the Data
[X] List data you need and how much is needed

    - Historical World Cup data for wins and losses?
    - SPI of all starters on national teams

[ ] Find and document where you get data

[ ] Get the data

[ ] Convert the data to a format you can manipulate

[ ] Check size and type of data (time series, sample, geographial, etc.)

[ ] Sample a test set, put it aside, and don't look at it. 


## Explore the data to gain insights
Note: try to get insights from a field expert for these steps.

[ ] Create a copy of the data for exploration

[ ] Create a jupyter notebook to keep record of data 
exploration

[ ] Study each attribute and its characteristics:

    - Name    
    - Type (categorical, int/float, bount/unbounded, text, structured, etc.)
            - .info(), .describe(), .shape, .head()        
    - Noisiness and type of noise (stochastic, outliers, rounding errors, etc.   
    - Possibly useful for the task?
    - Type of distribution (Gaussian, uniform, logarithmic, etc.)

[ ] For supervised learning, Identify the target attribute(s)

[ ] Visualize the data.

[ ] Study the correlations between attributes

[ ] Identify promising transformations you may want to apply. 

[ ] Document what you have learned
    

## Prepare the data to better expose the underlying data patterns to ML algorithms
Notes: 

    - Work on copies of data (Keep the original dataset intact).
    - Write functions for all data transformation you apply, for 5 reasons:
        1. You can easily prepare the data the next time you get a fresh dataset
        2. ability to apply these transformations in future projects
        3. To clean and prepare the test set
        4. To clean and new data instances
        5. To make it easy to treate your preparation choices as hyperparamteres
 

[ ] **Data Clearning**:

    - Fix or remove outliers (optional)
    - Fill in missing values (e.g., with zero, mean, median, etc.) or drop rows (columns)

[ ] Feature Selection (optional)

    - Drop the attributes that provide no useful information for the task.

[ ] **Feature engineering**, where appropriate:

    - Discretize continuous features.
    - Decompose features (e.g., categorical, date/time, etc.)
    - Aggregate features into promising new features.

[ ] **Feature Scaling** 

    - Standardize or normalize features


## Explore many different models and short-list the best ones
note: try to automate these steps as much as possible.


[ ] Train many quick and dirty models from different categories, using standard parameters.

    - linear
    - naive
    - Bayes
    - SVM
    - Random Forest
    - Neural net
    - etc.

[ ] Measure and compare their performance

    - For each model, use **N-fold cross-validation** and compute the standard deviation of their performance.

[ ] Analyze the most significant variables for each algorithm

[ ] Analyze the types of errors the models make

[ ] Quick round of feature selection and engineering

[ ] One or two more quick iterations of the 5 previous steps

[ ] short-list the top three to five most promising models, preferring models that make different types of errors


## Fine-tune your models and combine them into a solution.
Note: 

    - you will want to use as much data as possible for this step, especially as you move toward the end of fine-tuning.
    - automate what you can
    

[ ] Find-tune hyperparameters using **cross-validation**

    - treat your data transformation choices asa hyperparameters, especially when you are note sure about them (e.g., should I replace missing values with zero or the median value, etc.)
    - Unless there are very few hyperparameter values to explore, prefer random search over grid search. If training is very long, you might prefer a Bayesian optimization approach(e.g., a gaussian process priors)

[ ] Try **Ensemble methods**. Combining your best models will often perform better than running them indiviually.

[ ] Once you are confident about your final model, measure its performance on the test set to estimate the generalization error. 

**Note: Don't tweak your model after measuring the generalization error: you would just start overfitting the test set.**


## Present solution

[ ] Document what you have done

[ ] Create Presentation


## Launch, monitor, and Maintain

### https://www.kaggle.com/code/launay10christian/world-cup-prediction/notebook

Good source for baseline

## Glossary

**SPI** - rating designed to provide the best possible objective representation of a team's current overall skill level.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

## World Cup Matches Data

In [2]:
matches = pd.read_csv('wc_matches.csv')

In [3]:
matches.columns

Index(['date', 'league_id', 'league', 'team1', 'team2', 'spi1', 'spi2',
       'prob1', 'prob2', 'probtie', 'proj_score1', 'proj_score2', 'score1',
       'score2', 'xg1', 'xg2', 'nsxg1', 'nsxg2', 'adj_score1', 'adj_score2'],
      dtype='object')

In [4]:
matches.head()

,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,probtie,proj_score1,proj_score2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
0,2018-06-14,1908,FIFA World Cup,Russia,Saudi Arabia,70.14,51.33,0.6615,0.1200,0.2184,2.03,0.73,5,0,1.62,0.20,1.55,0.11,4.18,0.00
1,2018-06-15,1908,FIFA World Cup,Egypt,Uruguay,62.46,79.52,0.1845,0.5433,0.2723,0.82,1.61,0,1,0.42,1.94,0.65,1.94,0.00,1.05
2,2018-06-15,1908,FIFA World Cup,Morocco,Iran,72.33,65.57,0.4004,0.2667,0.3330,1.13,0.86,0,1,0.86,0.72,0.57,0.36,0.00,1.05
3,2018-06-15,1908,FIFA World Cup,Portugal,Spain,83.47,91.30,0.2459,0.4809,0.2732,1.07,1.60,3,3,1.15,1.63,0.80,2.59,3.15,3.15
4,2018-06-16,1908,FIFA World Cup,France,Australia,87.18,61.69,0.7694,0.0746,0.1560,2.61,0.69,2,1,1.38,1.03,1.24,0.39,2.10,1.05


In [6]:
#Adding goal difference and establishing who is the winner 
winner = []
for i in range (len(matches['team2'])):
    if matches ['score2'][i] > matches['score1'][i]:
        winner.append(matches['team2'][i])
    elif matches['score2'][i] < matches ['score1'][i]:
        winner.append(matches['team1'][i])
    else:
        winner.append('Draw')
matches['winning_team'] = winner

#adding goal difference column
matches['goal_difference'] = np.absolute(matches['score2'] - matches['score1'])

matches.head()

,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,probtie,...,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2,winning_team,goal_difference
0,2018-06-14,1908,FIFA World Cup,Russia,Saudi Arabia,70.14,51.33,0.6615,0.1200,0.2184,...,5,0,1.62,0.20,1.55,0.11,4.18,0.00,Russia,5
1,2018-06-15,1908,FIFA World Cup,Egypt,Uruguay,62.46,79.52,0.1845,0.5433,0.2723,...,0,1,0.42,1.94,0.65,1.94,0.00,1.05,Uruguay,1
2,2018-06-15,1908,FIFA World Cup,Morocco,Iran,72.33,65.57,0.4004,0.2667,0.3330,...,0,1,0.86,0.72,0.57,0.36,0.00,1.05,Iran,1
3,2018-06-15,1908,FIFA World Cup,Portugal,Spain,83.47,91.30,0.2459,0.4809,0.2732,...,3,3,1.15,1.63,0.80,2.59,3.15,3.15,Draw,0
4,2018-06-16,1908,FIFA World Cup,France,Australia,87.18,61.69,0.7694,0.0746,0.1560,...,2,1,1.38,1.03,1.24,0.39,2.10,1.05,France,1


In [ ]:
kjh

In [ ]:
matches.shape

## World Cup Comparisons Data

In [7]:
comparisons = pd.read_csv('world_cup_comparisons.csv')

In [8]:
comparisons.columns

Index(['player', 'season', 'team', 'goals_z', 'xg_z', 'crosses_z',
       'boxtouches_z', 'passes_z', 'progpasses_z', 'takeons_z', 'progruns_z',
       'tackles_z', 'interceptions_z', 'clearances_z', 'blocks_z', 'aerials_z',
       'fouls_z', 'fouled_z', 'nsxg_z'],
      dtype='object')

In [20]:
comparisons.tail()

,player,season,team,goals_z,xg_z,crosses_z,boxtouches_z,passes_z,progpasses_z,takeons_z,progruns_z,tackles_z,interceptions_z,clearances_z,blocks_z,aerials_z,fouls_z,fouled_z,nsxg_z
5894,Uwe Seeler,1966,West Germany,1.28,3.01,-0.08,4.34,0.04,-0.65,-0.42,-0.03,-0.68,-0.01,-0.63,-0.26,2.90,-0.09,2.14,0.38
5895,Werner Krämer,1966,West Germany,-0.39,-0.63,0.46,-0.51,-1.53,-1.11,-0.60,-0.73,-1.10,-0.85,-0.69,-0.62,-0.60,-1.21,-0.34,-0.17
5896,Willi Schulz,1966,West Germany,-0.39,-0.63,-0.35,-0.67,0.49,0.41,-0.77,-0.66,1.48,0.70,2.45,3.12,0.15,0.10,-0.51,-0.60
5897,Wolfgang Overath,1966,West Germany,-0.39,-0.12,0.73,0.05,1.82,0.87,0.11,0.26,1.64,1.49,-0.46,-0.62,-0.35,1.60,0.55,0.41
5898,Wolfgang Weber,1966,West Germany,0.45,-0.16,-0.62,-0.41,-0.55,-0.41,-0.60,-0.52,0.17,1.77,2.05,0.46,1.65,-0.09,1.25,-0.86


In [10]:
comparisons.shape

(5899, 19)

### Change comparisons to world cups 2018, 2014, and 2010

In [21]:
years = [2018, 2014, 2010]

In [27]:
comparisons = comparisons.loc[comparisons['season'].isin(years)]

In [28]:
comparisons.head()

,player,season,team,goals_z,xg_z,crosses_z,boxtouches_z,passes_z,progpasses_z,takeons_z,progruns_z,tackles_z,interceptions_z,clearances_z,blocks_z,aerials_z,fouls_z,fouled_z,nsxg_z
0,Cristian Pavón,2018,Argentina,-0.42,-0.55,0.08,-0.24,-0.61,-0.72,-0.05,0.15,0.30,-0.79,-0.80,-0.45,-0.23,-0.38,-0.91,-0.58
1,Eduardo Salvio,2018,Argentina,-0.42,-0.50,-0.51,0.64,0.02,-0.46,-0.05,0.48,0.61,0.67,0.32,-0.45,0.21,-1.11,0.68,-0.03
2,Enzo Pérez,2018,Argentina,-0.42,0.11,-0.51,-0.69,0.27,-0.65,-0.36,-0.34,0.35,-0.22,-0.80,-0.45,-0.89,0.72,0.04,-0.66
3,Federico Fazio,2018,Argentina,-0.42,-0.59,-0.51,-0.69,-0.97,-0.91,-0.67,-0.84,-0.82,-0.79,-0.64,-0.45,0.21,-1.11,-0.59,-0.77
4,Franco Armani,2018,Argentina,-0.42,-0.59,-0.51,-0.80,-0.67,0.64,-0.67,-0.67,-0.82,-0.79,-0.64,-0.45,-0.89,-1.11,-0.91,-0.77


In [29]:
comparisons.tail()

,player,season,team,goals_z,xg_z,crosses_z,boxtouches_z,passes_z,progpasses_z,takeons_z,progruns_z,tackles_z,interceptions_z,clearances_z,blocks_z,aerials_z,fouls_z,fouled_z,nsxg_z
1663,Nicolas Lodeiro,2010,Uruguay,-0.40,-0.63,-0.52,-0.52,-1.20,-1.03,-0.41,-0.56,-0.86,-0.57,-0.47,-0.22,-0.28,-0.43,-0.84,-0.64
1664,Sebastián Abreu,2010,Uruguay,-0.40,-0.63,-0.52,-0.29,-1.31,-1.09,-0.65,-0.93,-0.86,-0.76,-0.54,-0.21,-0.52,-0.43,-1.01,-0.65
1665,Walter Gargano,2010,Uruguay,-0.40,-0.63,-0.52,-0.75,-1.01,-1.00,-0.41,-0.83,-0.66,-0.67,-0.74,-0.48,-0.77,-0.76,-0.67,-0.76
1666,Álvaro Fernández,2010,Uruguay,-0.40,-0.58,-0.52,-0.60,-1.31,-1.09,-0.41,-0.93,-0.69,-0.80,-0.61,-0.48,-0.77,-0.76,-0.50,-0.81
1667,Álvaro Pereira,2010,Uruguay,0.54,0.28,0.07,-0.14,-0.35,-0.05,-0.65,-0.38,-0.18,-0.28,-0.07,-0.21,-0.52,-0.43,0.01,-0.20


## World Cup Data from 2018 - 1970

initialized to variable df

In [16]:
# df = pd.concat(
#     map(pd.read_csv, ['FIFA - 2018.csv', 'FIFA - 2014.csv', 'FIFA - 2010.csv', 'FIFA - 2006.csv', 'FIFA - 2002.csv', 'FIFA - 1998.csv', 'FIFA - 1994.csv', 'FIFA - 1990.csv', 'FIFA - 1986.csv', 'FIFA - 1982.csv', 'FIFA - 1978.csv', 'FIFA - 1974.csv', 'FIFA - 1970.csv' ]), ignore_index=True)
# print(df)

df = pd.concat(
    map(pd.read_csv, ['FIFA - 2018.csv', 'FIFA - 2014.csv', 'FIFA - 2010.csv']))


In [17]:
df.columns

Index(['Position', 'Team', 'Games Played', 'Win', 'Draw', 'Loss', 'Goals For',
       'Goals Against', 'Goal Difference', 'Points'],
      dtype='object')

In [18]:
df.shape

(96, 10)

In [19]:
df.head()

,Position,Team,Games Played,Win,Draw,Loss,Goals For,Goals Against,Goal Difference,Points
0,1,France,7,6,1,0,14,6,8,19
1,2,Croatia,7,4,2,1,14,9,5,14
2,3,Belgium,7,6,0,1,16,6,10,18
3,4,England,7,3,1,3,12,8,4,10
4,5,Uruguay,5,4,0,1,7,3,4,12
